In [2]:
import os
import glob
import numpy as np
import nibabel as nib
from pathlib import Path
import json
from tqdm import tqdm

def create_dataset_structure():
    """Create the required directory structure"""
    base_dir = "nnUNet_raw/Dataset001_Microtubules"
    for subdir in ['imagesTr', 'labelsTr']:
        Path(f"{base_dir}/{subdir}").mkdir(parents=True, exist_ok=True)
    return base_dir

def normalize_image(img):
    return img.astype(np.float32)

def convert_sdt_to_binary(sdt):
    """Convert signed distance transform to binary mask
    SDT range is [-1, 1], where negative values are inside the object"""
    return (sdt <= 0.0001).astype(np.uint8)

def convert_to_nifti(source_dir, target_dir, is_label=False, file_prefix=""):
    """Convert .npy files to .nii.gz files"""
    npy_files = sorted(glob.glob(os.path.join(source_dir, "*.npy")))
    converted_files = []
    
    for idx, npy_path in enumerate(tqdm(npy_files, desc="Converting files")):
        # Load and process data
        data = np.load(npy_path)
        
        if is_label:
            # Convert SDT to binary mask
            data = convert_sdt_to_binary(data)
        else:
            # Normalize image data
            data = normalize_image(data)
        
        # Add channel axis for nnUNet
        data = data[np.newaxis, ...]
        
        # Create NIFTI file
        filename = f"{file_prefix}{idx:03d}.nii.gz"
        nifti_path = os.path.join(target_dir, filename)
        
        # Create and save NIFTI file with correct orientation
        nifti_img = nib.Nifti1Image(data, affine=np.eye(4))
        nib.save(nifti_img, nifti_path)
        
        converted_files.append(filename)
    
    return converted_files

def create_dataset_json(base_dir, image_files, num_training):
    """Create dataset.json file"""
    dataset_json = {
        "name": "Microtubules",
        "description": "Microtubule segmentation in FIBSEM data",
        "reference": "",
        "licence": "",
        "release": "0.0",
        "tensorImageSize": "4D",
        "modality": {
            "0": "FIBSEM"
        },
        "labels": {
            "0": "background",
            "1": "microtubule"
        },
        "numTraining": num_training,
        "numTest": 0,
        "training": [],
        "test": []
    }
    
    # Add training cases
    for image_file in image_files:
        label_file = image_file.replace('image_', 'label_')
        dataset_json["training"].append({
            "image": f"imagesTr/{image_file}",
            "label": f"labelsTr/{label_file}"
        })
    
    # Save dataset.json
    with open(os.path.join(base_dir, "dataset.json"), 'w') as f:
        json.dump(dataset_json, f, indent=4)

def main():
    # Create directory structure
    base_dir = create_dataset_structure()
    
    # Convert images
    print("Converting images...")
    image_files = convert_to_nifti(
        source_dir="training/subvols/image",
        target_dir=f"{base_dir}/imagesTr",
        is_label=False,
        file_prefix="image_"
    )
    
    # Convert labels
    print("Converting labels...")
    convert_to_nifti(
        source_dir="training/labeled_sdt",
        target_dir=f"{base_dir}/labelsTr",
        is_label=True,
        file_prefix="label_"
    )
    
    # Create dataset.json
    print("Creating dataset.json...")
    create_dataset_json(base_dir, image_files, num_training=len(image_files))
    
    print(f"\nDataset conversion complete! Files are in {base_dir}")
    print("Next steps:")
    print("1. Run: nnUNetv2_plan_and_preprocess -d 1 --verify_dataset_integrity")
    print("2. Run your training script")

if __name__ == "__main__":
    main()

Converting images...


Converting files: 100%|██████████| 48/48 [00:01<00:00, 30.40it/s]


Converting labels...


Converting files: 100%|██████████| 49/49 [00:00<00:00, 79.36it/s]

Creating dataset.json...

Dataset conversion complete! Files are in nnUNet_raw/Dataset001_Microtubules
Next steps:
1. Run: nnUNetv2_plan_and_preprocess -d 1 --verify_dataset_integrity
2. Run your training script
